In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

from sklearn.model_selection import train_test_split

from tqdm import tqdm

from os import makedirs
from os.path import expanduser, exists, join

In [23]:
batch_size = 16
img_width, img_height = 256, 256
num_classes = 2
n_epoches = 100
input_shape = (img_height, img_width, 3)

In [24]:
train_data_dir = "../data/train"
valid_data_dir = "../data/valid"

In [59]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                featurewise_center=False,  # set input mean to 0 over the dataset
                                samplewise_center=True,  # set each sample mean to 0
                                featurewise_std_normalization=False,  # divide inputs by std of the dataset
                                samplewise_std_normalization=True,  # divide each input by its std
#                                 zca_whitening=False,  # apply ZCA whitening
#                                 rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
                                width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
                                height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
                                horizontal_flip=True,  # randomly flip images
                                vertical_flip=True) # randomly flip images                                

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    seed=10)

val_datagen = ImageDataGenerator(rescale=1. / 255)

valid_generator = val_datagen.flow_from_directory(valid_data_dir,
                                                target_size=(img_height, img_width),
                                                batch_size=batch_size,
                                                seed=10)

print ("train data: ", train_generator.n)
print ("valid data: ", valid_generator.n)

Found 204 images belonging to 2 classes.
Found 46 images belonging to 2 classes.
('train data: ', 204)
('valid data: ', 46)


In [63]:
def create_baseline():
    # create model  
    model = Sequential()
    BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None)
    model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=input_shape, padding = 'same'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(64, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(128, kernel_size=(3, 3),activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(256, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))

    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(BatchNormalization(epsilon=1e-06, mode=0, momentum=0.9, weights=None))  
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(num_classes, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(GlobalAveragePooling2D())
    
    model.add(Dense(num_class, activation='softmax'))
    return model

In [64]:
base_model = create_baseline()
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_78 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
batch_normalization_75 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
conv2d_79 (Conv2D)           (None, 256, 256, 64)      18496     
_________________________________________________________________
max_pooling2d_58 (MaxPooling (None, 128, 128, 64)      0         
_________________________________________________________________
batch_normalization_76 (Batc (None, 128, 128, 64)      256       
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_59 (MaxPooling (None, 64, 64, 128)       0         
__________

In [65]:
base_model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
base_model.save_weights("jbm_custom_weights_15.h5")

In [ ]:
base_model.fit_generator(train_generator,
                      steps_per_epoch = train_generator.n // batch_size,
                      validation_data = valid_generator,
                      validation_steps = valid_generator.n // batch_size,
                      epochs = n_epoches,
                      verbose = 1)

Epoch 1/100
12/12 [==============================] - 71s 6s/step - loss: 0.8598 - acc: 0.4723 - val_loss: 0.9241 - val_acc: 0.4375
Epoch 2/100
12/12 [==============================] - 66s 5s/step - loss: 0.6933 - acc: 0.4932 - val_loss: 0.9617 - val_acc: 0.4375
Epoch 3/100
12/12 [==============================] - 61s 5s/step - loss: 0.6969 - acc: 0.4880 - val_loss: 0.6913 - val_acc: 0.5000
Epoch 4/100
12/12 [==============================] - 61s 5s/step - loss: 0.6925 - acc: 0.5277 - val_loss: 0.6928 - val_acc: 0.5625
Epoch 5/100
12/12 [==============================] - 62s 5s/step - loss: 0.6925 - acc: 0.5729 - val_loss: 0.6928 - val_acc: 0.5625
Epoch 6/100
12/12 [==============================] - 59s 5s/step - loss: 0.6909 - acc: 0.5523 - val_loss: 0.6927 - val_acc: 0.5625
Epoch 7/100
12/12 [==============================] - 61s 5s/step - loss: 0.6860 - acc: 0.5229 - val_loss: 0.6926 - val_acc: 0.5625
Epoch 8/100
12/12 [==============================] - 62s 5s/step - loss: 0.6692 - a

In [ ]:
base_model.save_weights("jbm_inception_weights_5.h5")